In [1]:
import mlflow

# Set the MLflow tracking URI to a new SQLite URI
mlflow.set_tracking_uri("sqlite:///new_mlflow.db")
mlflow.set_experiment("AutoGluon")

<Experiment: artifact_location='/Users/arham/Downloads/Projects/03-Experiments/mlruns/3', creation_time=1713916994501, experiment_id='3', last_update_time=1713916994501, lifecycle_stage='active', name='AutoGluon', tags={}>

Experiment Setup

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
import lightgbm as lgb
from sklearn.metrics import accuracy_score

import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import warnings
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score


def load_data(path):
    df = pd.read_csv(path)
    train_df, test_df = train_test_split(df, test_size=0.35, random_state=42)
    train_df, val_df,  = train_test_split(train_df, test_size=0.20, random_state=42)
    train_df = train_df.drop(['id'], axis=1).drop_duplicates().reset_index(drop=True)
    test_df = test_df.drop(['id'], axis=1).drop_duplicates().reset_index(drop=True)
    val_df = val_df.drop(['id'], axis=1).drop_duplicates().reset_index(drop=True)
    return train_df, val_df, test_df

def encode_target(train):
    target_key = {'Insufficient_Weight': 0, 'Normal_Weight': 1, 'Overweight_Level_I': 2, 'Overweight_Level_II': 3, 'Obesity_Type_I': 4,'Obesity_Type_II' : 5, 'Obesity_Type_III': 6}
    train['NObeyesdad'] = train['NObeyesdad'].map(target_key)
    return train

def make_gender_binary(train):
    train['Gender'] = train['Gender'].map({'Male':0, 'Female':1})

def datatypes(train):
    train['Weight'] = train['Weight'].astype(float)
    train['Age'] = train['Age'].astype(float)
    train['Height'] = train['Height'].astype(float)
    return train

# def age_binning(train_df):
#     # train_df['Age_Group'] = pd.cut(train_df['Age'], bins=[0, 20, 30, 40, 50, train_df['Age'].max()], labels=['0-20', '21-30', '31-40', '41-50', '50+'])
#     train_df['Age_Group'] = pd.cut(train_df['Age'], bins=[0, 20, 30, 40, 50, train_df['Age'].max()], labels=[1, 2, 3, 4, 5])
#     train_df['Age_Group'] = train_df['Age_Group'].astype(int)
#     return train_df

def age_binning(df):
    age_groups = []
    for age in df['Age']:
        if age <= 20:
            age_group = 1
        elif age <= 30:
            age_group = 2
        elif age <= 40:
            age_group = 3
        elif age <= 50:
            age_group = 4
        else:
            age_group = 5
        age_groups.append(age_group)
    df['Age_Group'] = age_groups
    return df

def age_scaling_log(train_df):
    train_df['Age'] = train_df['Age'].astype(float)
    train_df['Log_Age'] = np.log1p(train_df['Age'])
    return train_df

def age_scaling_minmax(train_df):
    train_df['Age'] = train_df['Age'].astype(float)
    scaler_age = MinMaxScaler()
    train_df['Scaled_Age'] = scaler_age.fit_transform(train_df['Age'].values.reshape(-1, 1))
    return train_df, scaler_age

def weight_scaling_log(train_df):
    train_df['Weight'] = train_df['Weight'].astype(float)
    train_df['Log_Weight'] = np.log1p(train_df['Weight'])
    return train_df

def weight_scaling_minmax(train_df):
    train_df['Weight'] = train_df['Weight'].astype(float)
    scaler_weight = MinMaxScaler()
    train_df['Scaled_Weight'] = scaler_weight.fit_transform(train_df['Weight'].values.reshape(-1, 1))
    return train_df, scaler_weight

def height_scaling_log(train_df):
    train_df['Log_Height'] = np.log1p(train_df['Height'])
    return train_df

def height_scaling_minmax(train_df):
    scaler_height = MinMaxScaler()
    train_df['Scaled_Height'] = scaler_height.fit_transform(train_df['Height'].values.reshape(-1, 1))
    return train_df, scaler_height

def make_gender_binary(train):
    train['Gender'] = train['Gender'].map({'Female':1, 'Male':0})
    return train

def fix_binary_columns(train):
    Binary_Cols = ['family_history_with_overweight','FAVC', 'SCC','SMOKE']
    # if yes then 1 else 0
    for col in Binary_Cols:
        train[col] = train[col].map({'yes': 1, 'no': 0})
        # column datatype integer
        train[col] = train[col].astype(int)
    return train

def freq_cat_cols(train):
    # One hot encoding
    cat_cols = ['CAEC', 'CALC']
    for col in cat_cols:
        train[col] = train[col].map({'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3})
    return train

def Mtrans(train):
    """
    Public_Transportation    8692
    Automobile               1835
    Walking                   231
    Motorbike                  19
    Bike                       16
    """
    # train['MTRANS'] = train['MTRANS'].map({'Public_Transportation': 3, 'Automobile': 5, 'Walking': 1, 'Motorbike': 4, 'Bike': 2})
    # dummify column
    train = pd.get_dummies(train, columns=['MTRANS'])
    # convert these columns to integer
    train['MTRANS_Automobile'] = train['MTRANS_Automobile'].astype(int)
    train['MTRANS_Walking'] = train['MTRANS_Walking'].astype(int)
    train['MTRANS_Motorbike'] = train['MTRANS_Motorbike'].astype(int)
    train['MTRANS_Bike'] = train['MTRANS_Bike'].astype(int)
    train['MTRANS_Public_Transportation'] = train['MTRANS_Public_Transportation'].astype(int)
    return train


def other_features(train):
    train['BMI'] = train['Weight'] / (train['Height'] ** 2)
    # train['Age'*'Gender'] = train['Age'] * train['Gender']
    polynomial_features = PolynomialFeatures(degree=2)
    X_poly = polynomial_features.fit_transform(train[['Age', 'BMI']])
    poly_features_df = pd.DataFrame(X_poly, columns=['Age^2', 'Age^3', 'BMI^2', 'Age * BMI', 'Age * BMI^2', 'Age^2 * BMI^2'])
    train = pd.concat([train, poly_features_df], axis=1)
    return train


def test_pipeline(test, scaler_age, scaler_weight, scaler_height):
    test = datatypes(test)
    test = encode_target(test)
    test = age_binning(test)
    test = age_scaling_log(test)
    test['Scaled_Age'] = scaler_age.transform(test['Age'].values.reshape(-1, 1))
    test = weight_scaling_log(test)
    test['Scaled_Weight'] = scaler_weight.transform(test['Weight'].values.reshape(-1, 1))
    test = height_scaling_log(test)
    test['Scaled_Height'] = scaler_height.transform(test['Height'].values.reshape(-1, 1))
    test = make_gender_binary(test)
    test = fix_binary_columns(test)
    test = freq_cat_cols(test)
    test = Mtrans(test)
    test = other_features(test)

    return test

def train_model(params, X_train, y_train):
    lgb_train = lgb.Dataset(X_train, y_train)
    model = lgb.train(params, lgb_train, num_boost_round=1000)
    return model

def evaluate_model(model, X_val, y_val):
    y_pred = model.predict(X_val)
    y_pred = [np.argmax(y) for y in y_pred]
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

def objective(trial, X_train, y_train):
    params = {
        'objective': 'multiclass',
        'num_class': 7,
        'metric': 'multi_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.6, 0.95),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 0.95),
        'verbosity': -1
    }

    n_splits = 5
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []

    for train_index, val_index in kf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

        model = train_model(params, X_tr, y_tr)
        accuracy = evaluate_model(model, X_val, y_val)
        scores.append(accuracy)

    return np.mean(scores)

def optimize_hyperparameters(X_train, y_train, n_trials=2):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=n_trials)
    return study.best_params

Experiment

In [13]:
path = '/Users/arham/Downloads/Projects/01-Dataset/01-Data-for-model-building/train.csv'
train_df, val_df, test_df = load_data(path)

train_df = datatypes(train_df)
train_df = encode_target(train_df)
train_df = age_binning(train_df)
train_df, scaler_age = age_scaling_minmax(train_df)
train_df = age_scaling_log(train_df)
train_df, scaler_weight = weight_scaling_minmax(train_df)
train_df = weight_scaling_log(train_df)
train_df, scaler_height = height_scaling_minmax(train_df)
train_df = height_scaling_log(train_df)
train_df = make_gender_binary(train_df)
train_df = fix_binary_columns(train_df)
train_df = freq_cat_cols(train_df)
train_df = Mtrans(train_df)
train_df = other_features(train_df)

val_df = test_pipeline(val_df, scaler_age, scaler_weight, scaler_height)
test_df = test_pipeline(test_df, scaler_age, scaler_weight, scaler_height)

Target = 'NObeyesdad'
features = train_df.columns.drop(Target)

features = ['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 
       'MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike',
       'MTRANS_Public_Transportation', 'MTRANS_Walking']
       #   'BMI', 'Age^2',
       # 'Age^3', 'BMI^2', 'Age * BMI', 'Age * BMI^2', 'Age^2 * BMI^2'] 
#'Scaled_Age', 'Log_Age', 'Scaled_Weight', 'Log_Weight', 'Scaled_Height', 'Log_Height',

X_train = train_df[features]
y_train = train_df[Target]
X_val = val_df[features]
y_val = val_df[Target]
X_test = test_df[features]
y_test = test_df[Target]

#combine X_train and y_train as one dataframe
tr = pd.concat([X_train, y_train], axis=1)
te = pd.concat([X_test, y_test], axis =1)
va = pd.concat([X_val, y_val], axis = 1)

In [16]:
y_train.value_counts()/len(y_train)
# to dict as weights
weights = y_train.value_counts(normalize=True).to_dict()
weights

{6: 0.19790605021773372,
 5: 0.1545446122486797,
 1: 0.15056054850365977,
 4: 0.13879366255906606,
 0: 0.12202353377188918,
 3: 0.12081904938385991,
 2: 0.11535254331511165}

In [10]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.metrics import precision_recall_fscore_support
import mlflow

# Load your data into AutoGluon TabularDataset format
train_data = TabularDataset(X_train.join(y_train))
val_data = TabularDataset(X_val.join(y_val))

# Define the label column
label_column = Target # Replace 'Target' with your actual label column name

# Specify the task and run AutoGluon
predictor = TabularPredictor(label=label_column).fit(train_data=train_data)

# Make predictions on the validation set
y_val_pred_autogluon = predictor.predict(val_data.drop(columns=[label_column]))

# Evaluate performance
precision, recall, f1, support = precision_recall_fscore_support(y_val, y_val_pred_autogluon, average='weighted')

# Log metrics and model using MLflow
with mlflow.start_run(run_name="AutoGluon_Without_Feature_Engineering"):
    # Log AutoGluon model
    mlflow.sklearn.log_model(predictor, "autogluon_model")
    
    # Log metrics
    mlflow.log_metric('accuracy', accuracy_score(y_val, y_val_pred_autogluon))
    mlflow.log_metric('precision', precision)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('f1', f1)

    # Log recall per class
    recall_per_class = recall_score(y_val, y_val_pred_autogluon, average=None)
    for i, recall_class in enumerate(recall_per_class):
        print(f"Recall for class {i}: {recall_class}")
        mlflow.log_metric(f'recall_class_{i}', recall_class)

    mlflow.set_tag('experiments', 'Arham A.')
    mlflow.set_tag('model_name', 'AutoGluon')
    mlflow.set_tag('preprocessing', 'Yes')

No path specified. Models will be saved in: "AutogluonModels/ag-20240426_034247"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240426_034247"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.0.0: Fri Sep 15 14:42:57 PDT

Recall for class 0: 0.9462025316455697
Recall for class 1: 0.8957219251336899
Recall for class 2: 0.755223880597015
Recall for class 3: 0.8449848024316109
Recall for class 4: 0.8788598574821853
Recall for class 5: 0.9688995215311005
Recall for class 6: 0.9960474308300395


In [28]:
!mlflow ui --backend-store-uri "sqlite:////Users/arham/Downloads/Projects/03-Experiments/new_mlflow.db"

[2024-04-26 05:38:37 -0400] [14055] [INFO] Starting gunicorn 21.2.0
[2024-04-26 05:38:37 -0400] [14055] [INFO] Listening at: http://127.0.0.1:5000 (14055)
[2024-04-26 05:38:37 -0400] [14055] [INFO] Using worker: sync
[2024-04-26 05:38:37 -0400] [14056] [INFO] Booting worker with pid: 14056
[2024-04-26 05:38:37 -0400] [14057] [INFO] Booting worker with pid: 14057
[2024-04-26 05:38:37 -0400] [14058] [INFO] Booting worker with pid: 14058
[2024-04-26 05:38:37 -0400] [14059] [INFO] Booting worker with pid: 14059
[2024-04-26 06:53:19 -0400] [14055] [CRITICAL] WORKER TIMEOUT (pid:14056)
[2024-04-26 06:53:19 -0400] [14055] [CRITICAL] WORKER TIMEOUT (pid:14057)
[2024-04-26 06:53:19 -0400] [14055] [CRITICAL] WORKER TIMEOUT (pid:14058)
[2024-04-26 06:53:19 -0400] [14055] [CRITICAL] WORKER TIMEOUT (pid:14059)
[2024-04-26 06:53:19 -0400] [14059] [INFO] Worker exiting (pid: 14059)
[2024-04-26 06:53:19 -0400] [14058] [INFO] Worker exiting (pid: 14058)
[2024-04-26 06:53:19 -0400] [14057] [INFO] Worker

## Saving probabilities for stacked model later

In [8]:
# Make predictions on train, validation, and test data
y_pred_train = predictor.predict(tr)
y_pred_va = predictor.predict(va)
y_pred_test = predictor.predict(te)

# Get prediction probabilities for each class
y_probabilities_train = predictor.predict_proba(tr)
y_probabilities_va = predictor.predict_proba(va)
y_probabilities_test = predictor.predict_proba(te)

# add to tr, te, va
test = pd.concat([te, y_probabilities_test.iloc[:, -7:]], axis=1)
train = pd.concat([tr, y_probabilities_train.iloc[:, -7:]], axis=1)
val = pd.concat([va, y_probabilities_va.iloc[:, -7:]], axis=1)


In [9]:
train

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,Age * BMI^2,Age^2 * BMI^2,NObeyesdad,0,1,2,3,4,5,6
0,1,21.000000,1.550000,51.000000,0,1,3.000000,1.0,2,0,...,445.785640,450.623213,0,0.797175,0.196596,0.005617,0.000520,0.000032,0.000019,0.000040
1,0,20.000000,1.700000,80.000000,1,1,2.000000,3.0,1,0,...,553.633218,766.274350,3,0.000041,0.001926,0.047782,0.948167,0.002041,0.000018,0.000025
2,1,18.000000,1.600000,60.000000,1,1,2.000000,3.0,1,0,...,421.875000,549.316406,1,0.011628,0.953612,0.033047,0.001538,0.000150,0.000005,0.000020
3,1,26.000000,1.632983,111.720238,1,1,3.000000,3.0,1,0,...,1089.285877,1755.242193,6,0.000001,0.000002,0.000008,0.000018,0.000172,0.000040,0.999759
4,1,21.682636,1.748524,133.845064,1,1,3.000000,3.0,1,0,...,949.229536,1916.541944,6,0.000001,0.000001,0.000004,0.000012,0.000092,0.000058,0.999831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10788,0,18.000000,1.780000,108.000000,1,1,2.000000,3.0,1,0,...,613.558894,1161.896656,4,0.000007,0.000014,0.000035,0.000287,0.998704,0.000949,0.000005
10789,1,26.000000,1.641601,111.830924,1,1,3.000000,3.0,1,0,...,1078.946835,1722.080284,6,0.000001,0.000002,0.000007,0.000020,0.000154,0.000059,0.999757
10790,0,21.000000,1.770000,75.000000,0,1,3.000000,3.0,2,0,...,502.729101,573.098750,2,0.000212,0.341897,0.657261,0.000391,0.000052,0.000019,0.000168
10791,0,29.669219,1.774644,105.966894,1,1,2.934671,3.0,1,0,...,998.283353,1132.127734,4,0.000621,0.000067,0.000338,0.004232,0.936736,0.057409,0.000597


In [27]:
path = '/Users/arham/Downloads/Projects/01-Dataset/01-Data-for-model-building/train.csv'
train_df, val_df, test_df = load_data(path)

train_df = datatypes(train_df)
train_df = encode_target(train_df)
train_df = age_binning(train_df)
train_df, scaler_age = age_scaling_minmax(train_df)
train_df = age_scaling_log(train_df)
train_df, scaler_weight = weight_scaling_minmax(train_df)
train_df = weight_scaling_log(train_df)
train_df, scaler_height = height_scaling_minmax(train_df)
train_df = height_scaling_log(train_df)
train_df = make_gender_binary(train_df)
train_df = fix_binary_columns(train_df)
train_df = freq_cat_cols(train_df)
train_df = Mtrans(train_df)
train_df = other_features(train_df)

val_df = test_pipeline(val_df, scaler_age, scaler_weight, scaler_height)
test_df = test_pipeline(test_df, scaler_age, scaler_weight, scaler_height)

Target = 'NObeyesdad'
features = train_df.columns.drop(Target)

features = ['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'Age_Group', 
       'MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike',
       'MTRANS_Public_Transportation', 'MTRANS_Walking', 'BMI', 'Age^2',
       'Age^3', 'BMI^2', 'Age * BMI', 'Age * BMI^2', 'Age^2 * BMI^2', 
       'Scaled_Age', 'Log_Age', 'Scaled_Weight', 'Log_Weight', 'Scaled_Height', 'Log_Height']
#'Scaled_Age', 'Log_Age', 'Scaled_Weight', 'Log_Weight', 'Scaled_Height', 'Log_Height',

X_train = train_df[features]
y_train = train_df[Target]
X_val = val_df[features]
y_val = val_df[Target]
X_test = test_df[features]
y_test = test_df[Target]

#combine X_train and y_train as one dataframe
tr = pd.concat([X_train, y_train], axis=1)
te = pd.concat([X_test, y_test], axis =1)
va = pd.concat([X_val, y_val], axis = 1)


,Description,Value
0,Session id,5119
1,Target,NObeyesdad
2,Target type,Multiclass
3,Original data shape,"(10793, 35)"
4,Transformed data shape,"(10793, 35)"
5,Transformed train set shape,"(7555, 35)"
6,Transformed test set shape,"(3238, 35)"
7,Numeric features,34
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8307,0.9024,0.8307,0.8307,0.8304,0.8013,0.8014
1,0.8241,0.8991,0.8241,0.8259,0.8248,0.7937,0.7938
2,0.8571,0.9178,0.8571,0.8616,0.8588,0.8324,0.8325
3,0.8347,0.9048,0.8347,0.8336,0.8334,0.8059,0.8061
4,0.8254,0.8996,0.8254,0.8254,0.8253,0.7951,0.7951
5,0.8477,0.9123,0.8477,0.8471,0.8473,0.8213,0.8213
6,0.8278,0.9007,0.8278,0.8268,0.8271,0.7979,0.7980
7,0.8291,0.9019,0.8291,0.8330,0.8298,0.7997,0.8002
8,0.8424,0.9093,0.8424,0.8451,0.8434,0.8151,0.8152


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.8222,0.8973,0.8222,0.8227,0.8222,0.7916,0.7917


ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

### Zero Shot Learning

In [20]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.metrics import precision_recall_fscore_support
import mlflow

# Load your data into AutoGluon TabularDataset format
train_data = TabularDataset(X_train.join(y_train))
val_data = TabularDataset(X_val.join(y_val))

# Define the label column
label_column = Target  # Replace 'Target' with your actual label column name

# Specify the task and run AutoGluon
predictor = TabularPredictor(label=label_column).fit(train_data=train_data)

# Make predictions on the validation set
y_val_pred_autogluon = predictor.predict(val_data.drop(columns=[label_column]))

# Evaluate performance
precision, recall, f1, support = precision_recall_fscore_support(y_val, y_val_pred_autogluon, average='weighted')

# Log metrics and model using MLflow
with mlflow.start_run(run_name="AutoGluon_Without_Feature_Engineering"):
    # Log AutoGluon model
    mlflow.sklearn.log_model(predictor, "autogluon_model")
    
    # Log metrics
    mlflow.log_metric('accuracy', accuracy_score(y_val, y_val_pred_autogluon))
    mlflow.log_metric('precision', precision)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('f1', f1)

    # Log recall per class
    recall_per_class = recall_score(y_val, y_val_pred_autogluon, average=None)
    for i, recall_class in enumerate(recall_per_class):
        print(f"Recall for class {i}: {recall_class}")
        mlflow.log_metric(f'recall_class_{i}', recall_class)

    mlflow.set_tag('experiments', 'Arham A.')
    mlflow.set_tag('model_name', 'AutoGluon')
    mlflow.set_tag('preprocessing', 'Yes')


No path specified. Models will be saved in: "AutogluonModels/ag-20240426_052138"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240426_052138"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.0.0: Fri Sep 15 14:42:57 PDT

Recall for class 0: 0.9335443037974683
Recall for class 1: 0.8983957219251337
Recall for class 2: 0.7582089552238805
Recall for class 3: 0.8389057750759878
Recall for class 4: 0.8646080760095012
Recall for class 5: 0.9641148325358851
Recall for class 6: 0.9960474308300395
